In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
from utils import CIFAR10, CIFAR100
print("Reading dataset...")
train_data = CIFAR10(train=True)
test_data  = CIFAR10(train=False)

Xtrain, Ytrain = train_data.train_data, train_data.train_labels
Xtest, Ytest = test_data.test_data, test_data.test_labels

Reading dataset...


In [3]:
print("training dataset: %s",Xtrain.shape)

training dataset: %s (50000, 32, 32, 3)


In [4]:
"""
A pure TensorFlow implementation of a neural network. This can be
used as a drop-in replacement for a Keras model.
"""

import numpy as np
import tensorflow as tf
from cleverhans.model import Model


class MLP(Model):
    """
    An example of a bare bones multilayer perceptron (MLP) class.
    """

    def __init__(self, layers, input_shape, pretrain_dict=None):
        super(MLP, self).__init__()
        
        self.layer_names = []
        self.layers = layers
        self.input_shape = input_shape
            
        for i, layer in enumerate(self.layers):
            if hasattr(layer, 'name'):
                name = layer.name
                if (pretrain_dict is not None) and (name in pretrain_dict.keys()):
                    if isinstance(layer, BatchNormalization):
                        layer.set_input_shape(shape=input_shape,
                                              mean=pretrain_dict[name+'_bn_mean'],
                                              variance=pretrain_dict[name+'_bn_variance'], 
                                              gamma=pretrain_dict[name+'_gamma'],
                                              beta=pretrain_dict[name+'_beta'])
                    else:
                        layer.set_input_shape(input_shape,
                                              pretrain = pretrain_dict[name])
                else:
                    layer.set_input_shape(input_shape)
            else:
                name = layer.__class__.__name__ + str(i)
                layer.name = name
                layer.set_input_shape(input_shape)
            
            print(layer.input_shape)
            self.layer_names.append(name)
            input_shape = layer.get_output_shape()
        
        if isinstance(layers[-1], Softmax):
            layers[-1].name = 'probs'
            layers[-2].name = 'logits'
            self.layer_names[-1] = 'probs'
            self.layer_names[-2] = 'logits'
        else:
            layers[-1].name = 'logits'
            self.layer_names[-1] = 'logits'

    def fprop(self, x, set_ref=False):
        states = []
        for layer in self.layers:
            if set_ref:
                layer.ref = x
            x = layer.fprop(x)
            assert x is not None
            states.append(x)
        states = dict(zip(self.get_layer_names(), states))
        return states


class Layer(object):

    def get_output_shape(self):
        return self.output_shape


class Linear(Layer):

    def __init__(self, num_hid=None, **kwargs):
        self.__dict__.update(kwargs)
        self.num_hid = num_hid

    def set_input_shape(self, input_shape, pretrain=None):
        batch_size, dim = input_shape
        if pretrain is None:
            if self.num_hid is None: 
                print("Without pre-trained models, please specify num_hid in Linear Layer.")
            init = tf.random_normal([dim, self.num_hid], dtype=tf.float32)
            init = init / tf.sqrt(1e-7 + tf.reduce_sum(tf.square(init), axis=0, keep_dims=True))
            self.W = tf.Variable(init)
            self.b = tf.Variable(np.zeros((self.num_hid,)).astype('float32'))
            
        else:
            self.W = tf.Variable(initial_value = pretrain[0], dtype = tf.float32)
            self.b = tf.Variable(initial_value = pretrain[1], dtype = tf.float32)
            self.num_hid = len(pretrain[1]) 
        
        self.input_shape = [batch_size, dim]
        self.output_shape = [batch_size, self.num_hid]

    def fprop(self, x):
        return tf.matmul(x, self.W) + self.b


class Conv2D(Layer):

    def __init__(self, output_channels=None, kernel_shape=None, strides=[1,1], padding='SAME', **kwargs):
        self.__dict__.update(kwargs)
        self.__dict__.update(locals())
        del self.self

    def set_input_shape(self, input_shape, pretrain=None):
        batch_size, rows, cols, input_channels = input_shape
        if pretrain is None:
            #assert ((self.kernel_shape is None) or (self.output_channels is None)),\
            #        "Without a pre-trained model, please specify kernel_shape and output_channels in Conv2D layer"
        
            kernel_shape = tuple(self.kernel_shape) + (input_channels, self.output_channels)
            assert len(kernel_shape) == 4
            assert all(isinstance(e, int) for e in kernel_shape), kernel_shape

            init = tf.random_normal(kernel_shape, dtype=tf.float32)
            init = init / tf.sqrt(1e-7 + tf.reduce_sum(tf.square(init), axis=(0, 1, 2)))
            self.kernels = tf.Variable(init)
            self.b = tf.Variable(np.zeros((self.output_channels,)).astype('float32'))
        else:
            self.kernels = tf.Variable(initial_value = pretrain[0], dtype = tf.float32)
            self.b = tf.Variable(initial_value = pretrain[1], dtype = tf.float32)
            self.kernel_shape = tuple(pretrain[0].shape[:2])
            self.output_channels = tuple(pretrain[0].shape[2:3])
        self.input_shape = input_shape
        input_shape = list(input_shape)
        input_shape[0] = 1
        dummy_batch = tf.zeros(input_shape)
        dummy_output = self.fprop(dummy_batch)
        output_shape = [int(e) for e in dummy_output.get_shape()]
        output_shape[0] = batch_size
        self.output_shape = tuple(output_shape)

    def fprop(self, x):
        return tf.nn.conv2d(x, self.kernels, (1,) + tuple(self.strides) + (1,),
                            self.padding) + self.b


class ReLU(Layer):

    def __init__(self):
        pass

    def set_input_shape(self, shape):
        self.input_shape = shape
        self.output_shape = shape

    def fprop(self, x):
        return tf.nn.relu(x)


class Softmax(Layer):

    def __init__(self):
        pass

    def set_input_shape(self, shape):
        self.input_shape = shape
        self.output_shape = shape

    def fprop(self, x):
        return tf.nn.softmax(x)
    
class MaxPool(Layer):
    
    def __init__(self):
        pass
    
    def set_input_shape(self, shape):
        batch_size, rows, cols, input_channels = shape
        input_shape = list(shape)
        input_shape[0] = 1
        dummy_batch = tf.zeros(input_shape)
        dummy_output = self.fprop(dummy_batch)
        output_shape = [int(e) for e in dummy_output.get_shape()]
        output_shape[0] = batch_size
        
        self.input_shape = shape
        self.output_shape = tuple(output_shape)
        
    def fprop(self, x):
        return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')


class Flatten(Layer):

    def __init__(self):
        pass

    def set_input_shape(self, shape):
        self.input_shape = shape
        output_width = 1
        for factor in shape[1:]:
            output_width *= factor
        self.output_width = output_width
        self.output_shape = [shape[0], output_width]

    def fprop(self, x):
        return tf.reshape(x, [-1, self.output_width])
    
class BatchNormalization(Layer):
    
    def __init__(self, dp=1.0, is_train=False, **kwargs):
        self.__dict__.update(kwargs)
        self.__dict__.update(locals())
        del self.self
    
    def set_input_shape(self, shape, mean, variance, gamma, beta):
        self.input_shape = shape
        self.output_shape = shape
        self.gamma = tf.Variable(initial_value=gamma)
        self.beta = tf.Variable(initial_value=beta)
        self.mean = tf.Variable(initial_value=mean)
        self.variance = tf.Variable(initial_value=variance)
    
    def fprop(self, x):
        H,W,C,O = x.get_shape().as_list()
        n1 = int(O * self.dp)
        n0 = O - n1
        mask = tf.constant(value=np.append(np.ones(n1, dtype='float32'), np.zeros(n0, dtype='float32')), dtype=tf.float32)
        conv_gamma = tf.multiply(self.gamma, mask)
        beta = tf.multiply(self.beta, mask)
        
#         moving_mean = self.mean
#         moving_variance = self.variance
        
#         from tensorflow.python.training.moving_averages import assign_moving_average
#         def mean_var_with_update():
#             mean, variance = tf.nn.moments(x, [0,1,2], name='moments')
#             with tf.control_dependencies([assign_moving_average(tf.cast(moving_mean, tf.float32), mean, 0.9),
#                                           assign_moving_average(tf.cast(moving_variance, tf.float32), variance, 0.9)]):
#                 return tf.identity(mean), tf.identity(variance)

#         self.mean, self.variance = tf.cond(tf.cast(self.is_train, tf.bool), mean_var_with_update, lambda:(moving_mean, moving_variance))

        conv = tf.nn.batch_normalization(x, self.mean, self.variance, self.beta, conv_gamma, 1e-05)

        return conv


def make_basic_cnn(nb_filters=64, nb_classes=10,
                   input_shape=(None, 28, 28, 1)):
    layers = [Conv2D(output_channels=nb_filters, kernel_shape=(8, 8), strides=(2, 2), padding="SAME"),
              ReLU(),
              Conv2D(output_channels=nb_filters * 2, kernel_shape=(6, 6), strides=(2, 2), padding="SAME"),
              ReLU(),
              Conv2D(output_channels=nb_filters * 2, kernel_shape=(5, 5), strides=(1, 1), padding="SAME"),
              ReLU(),
              Flatten(),
              Linear(num_hid=nb_classes),
              Softmax()]

    model = MLP(layers, input_shape)
    return model

def make_pretrain_vgg16(input_shape=(None, 28, 28, 1), pretrain_dict=None):
    layers = [Conv2D(name='conv1_1'),
                ReLU(),
                Conv2D(name='conv1_2'),
                ReLU(),
                Conv2D(name='conv2_1'),
                ReLU(),
                Conv2D(name='conv2_2'),
                ReLU(),
                Conv2D(name='conv3_1'),
                ReLU(),
                Conv2D(name='conv3_2'),
                ReLU(),
                Conv2D(name='conv3_3'),
                ReLU(),
                Conv2D(name='conv4_1'),
                ReLU(),
                Conv2D(name='conv4_2'),
                ReLU(),
                Conv2D(name='conv4_3'),
                ReLU(),
                Conv2D(name='conv5_1'),
                ReLU(),
                Conv2D(name='conv5_2'),
                ReLU(),
                Conv2D(name='conv5_3'),
                ReLU(),
                Flatten(),
                Linear(name='fc6'),
                ReLU(),
                Linear(name='fc7'),
                ReLU(),
                Linear(num_hid=10),
                Softmax()]
    model = MLP(layers, input_shape,pretrain_dict)
    return model

def make_reduced_vgg16(input_shape=(None, 28, 28, 1), pretrain_dict=None):
    layers = [Conv2D(name='conv1_1'),
              BatchNormalization(name='conv1_1'),
              ReLU(),
              Conv2D(name='conv1_2'),
              BatchNormalization(name='conv1_2'),
              ReLU(),
              MaxPool(),
              Conv2D(name='conv2_1'),
              BatchNormalization(name='conv2_1'),
              ReLU(),
              Conv2D(name='conv2_2'),
              BatchNormalization(name='conv2_2'),
              ReLU(),
              MaxPool(),
              Conv2D(name='conv3_1'),
              BatchNormalization(name='conv3_1'),
              ReLU(),
              Conv2D(name='conv3_2'),
              BatchNormalization(name='conv3_2'),
              ReLU(),
              Conv2D(name='conv3_3'),
              BatchNormalization(name='conv3_3'),
              ReLU(),
              MaxPool(),
              Conv2D(name='conv4_1'),
              BatchNormalization(name='conv4_1'),
              ReLU(),
              Conv2D(name='conv4_2'),
              BatchNormalization(name='conv4_2'),
              ReLU(),
              Conv2D(name='conv4_3'),
              BatchNormalization(name='conv4_3'),
              ReLU(),
              MaxPool(),
              Conv2D(name='conv5_1'),
              BatchNormalization(name='conv5_1'),
              ReLU(),
              Conv2D(name='conv5_2'),
              BatchNormalization(name='conv5_2'),
              ReLU(),
              Conv2D(name='conv5_3'),
              BatchNormalization(name='conv5_3'),
              ReLU(),
              MaxPool(),
              Flatten(),
              Linear(name='fc_1'),
              ReLU(),
              Linear(name='fc_2'),
              Softmax()]
    model = MLP(layers, input_shape, pretrain_dict)
    return model
    

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [5]:
sparse = np.load("finetune_dict.npy",encoding='latin1').item()

In [6]:
full = np.load("para_dict.npy", encoding='latin1').item()

In [22]:
model = make_reduced_vgg16(input_shape=(None,32,32,3),pretrain_dict=sparse)

(None, 32, 32, 3)
(None, 32, 32, 36)
(None, 32, 32, 36)
(None, 32, 32, 36)
(None, 32, 32, 56)
(None, 32, 32, 56)
(None, 32, 32, 56)
(None, 16, 16, 56)
(None, 16, 16, 93)
(None, 16, 16, 93)
(None, 16, 16, 93)
(None, 16, 16, 105)
(None, 16, 16, 105)
(None, 16, 16, 105)
(None, 8, 8, 105)
(None, 8, 8, 150)
(None, 8, 8, 150)
(None, 8, 8, 150)
(None, 8, 8, 140)
(None, 8, 8, 140)
(None, 8, 8, 140)
(None, 8, 8, 150)
(None, 8, 8, 150)
(None, 8, 8, 150)
(None, 4, 4, 150)
(None, 4, 4, 116)
(None, 4, 4, 116)
(None, 4, 4, 116)
(None, 4, 4, 37)
(None, 4, 4, 37)
(None, 4, 4, 37)
(None, 4, 4, 17)
(None, 4, 4, 17)
(None, 4, 4, 17)
(None, 2, 2, 17)
(None, 2, 2, 17)
(None, 2, 2, 17)
(None, 2, 2, 17)
(None, 2, 2, 15)
(None, 2, 2, 15)
(None, 2, 2, 15)
(None, 2, 2, 36)
(None, 2, 2, 36)
(None, 2, 2, 36)
(None, 1, 1, 36)
[None, 36]
[None, 512]
[None, 512]
[None, 10]


In [32]:
model = make_reduced_vgg16(input_shape=(None,32,32,3),pretrain_dict=full)

(None, 32, 32, 3)
(None, 32, 32, 64)
(None, 32, 32, 64)
(None, 32, 32, 64)
(None, 32, 32, 64)
(None, 32, 32, 64)
(None, 32, 32, 64)
(None, 16, 16, 64)
(None, 16, 16, 128)
(None, 16, 16, 128)
(None, 16, 16, 128)
(None, 16, 16, 128)
(None, 16, 16, 128)
(None, 16, 16, 128)
(None, 8, 8, 128)
(None, 8, 8, 256)
(None, 8, 8, 256)
(None, 8, 8, 256)
(None, 8, 8, 256)
(None, 8, 8, 256)
(None, 8, 8, 256)
(None, 8, 8, 256)
(None, 8, 8, 256)
(None, 8, 8, 256)
(None, 4, 4, 256)
(None, 4, 4, 512)
(None, 4, 4, 512)
(None, 4, 4, 512)
(None, 4, 4, 512)
(None, 4, 4, 512)
(None, 4, 4, 512)
(None, 4, 4, 512)
(None, 4, 4, 512)
(None, 4, 4, 512)
(None, 2, 2, 512)
(None, 2, 2, 512)
(None, 2, 2, 512)
(None, 2, 2, 512)
(None, 2, 2, 512)
(None, 2, 2, 512)
(None, 2, 2, 512)
(None, 2, 2, 512)
(None, 2, 2, 512)
(None, 2, 2, 512)
(None, 1, 1, 512)
[None, 512]
[None, 512]
[None, 512]
[None, 10]


In [33]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import numpy as np
import tensorflow as tf
from tensorflow.python.platform import flags
import logging

from cleverhans.utils_tf import model_train, model_eval
from cleverhans.attacks import FastGradientMethod, BasicIterativeMethod
from cleverhans.utils import AccuracyReport, set_log_level

In [34]:
train_params = {
    'nb_epochs': 1,
    'batch_size': 32,
    'learning_rate': 1e-4
}

In [41]:
# Object used to keep track of (and return) key accuracies
report = AccuracyReport()

# Set TF random seed to improve reproducibility
tf.set_random_seed(1234)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    batch_size = 128

    # Define input TF placeholder
    x = tf.placeholder(tf.float32, shape=(None, 32, 32, 3))
    y = tf.placeholder(tf.float32, shape=(None, 10))

    model_path = "models/"
    
    fgsm_params = {'eps': 1,
                   'clip_min': 0.,
                   'clip_max': 255.}
    rng = np.random.RandomState([2017, 8, 30])

    preds = model.get_probs(x)
    eval_params = {'batch_size': batch_size}
    acc = model_eval(sess, x, y, preds, Xtest, Ytest, args=eval_params)
    report.clean_train_clean_eval = acc
    
    fgsm = FastGradientMethod(model, sess=sess)
    adv_x = fgsm.generate(x, **fgsm_params)
    preds_adv = model.get_probs(adv_x)

    eval_par = {'batch_size': batch_size}
    acc = model_eval(sess, x, y, preds_adv, Xtest, Ytest, args=eval_par)
    print('Test accuracy on adversarial examples: %0.4f\n' % acc)
    report.clean_train_adv_eval = acc

Test accuracy on adversarial examples: 0.6103



In [42]:
report.clean_train_clean_eval

0.8819

In [43]:
report.clean_train_adv_eval

0.6103

In [ ]:
#     bim = BasicIterativeMethod(model=model, sess=sess)
#     bim_params = {'eps':0.1,
#                   'eps_iter':1,
#                   'clip_min': 0.,
#                   'clip_max': 1.}
#     adv_x = bim.generate(x, **bim_params)

In [55]:
with tf.Session() as sess:
    model_2 = make_reduced_vgg16(input_shape=(None,32,32,3),pretrain_dict=sparse)
    preds_2 = model_2(x)
    fgsm2 = FastGradientMethod(model_2, sess=sess)
    adv_x_2 = fgsm2.generate(x, **fgsm_params)
    adv_x_2 = tf.stop_gradient(adv_x_2)
    preds_2_adv = model_2(adv_x_2)

    def evaluate_2():
        # Accuracy of adversarially trained model on legitimate test inputs
        eval_params = {'batch_size': batch_size}
        accuracy = model_eval(sess, x, y, preds_2, Xtest, Ytest,
                              args=eval_params)
        print('Test accuracy on legitimate examples: %0.4f' % accuracy)
        report.adv_train_clean_eval = accuracy

        # Accuracy of the adversarially trained model on adversarial examples
        accuracy = model_eval(sess, x, y, preds_2_adv, Xtest,
                              Ytest, args=eval_params)
        print('Test accuracy on adversarial examples: %0.4f' % accuracy)
        report.adv_train_adv_eval = accuracy

    # Perform and evaluate adversarial training
    model_train(sess, x, y, preds_2, Xtrain, Ytrain,
                predictions_adv=preds_2_adv, evaluate=evaluate_2,
                args=train_params, rng=rng)

    eval_params = {'batch_size': batch_size}
    accuracy = model_eval(sess, x, y, preds_2, Xtrain, Ytrain,
                          args=eval_params)
    report.train_adv_train_clean_eval = accuracy
    accuracy = model_eval(sess, x, y, preds_2_adv, Xtrain,
                          Ytrain, args=eval_params)
    report.train_adv_train_adv_eval = accuracy

(None, 32, 32, 3)
(None, 32, 32, 36)
(None, 32, 32, 36)
(None, 32, 32, 36)
(None, 32, 32, 56)
(None, 32, 32, 56)
(None, 32, 32, 56)
(None, 16, 16, 56)
(None, 16, 16, 93)
(None, 16, 16, 93)
(None, 16, 16, 93)
(None, 16, 16, 105)
(None, 16, 16, 105)
(None, 16, 16, 105)
(None, 8, 8, 105)
(None, 8, 8, 150)
(None, 8, 8, 150)
(None, 8, 8, 150)
(None, 8, 8, 140)
(None, 8, 8, 140)
(None, 8, 8, 140)
(None, 8, 8, 150)
(None, 8, 8, 150)
(None, 8, 8, 150)
(None, 4, 4, 150)
(None, 4, 4, 116)
(None, 4, 4, 116)
(None, 4, 4, 116)
(None, 4, 4, 37)
(None, 4, 4, 37)
(None, 4, 4, 37)
(None, 4, 4, 17)
(None, 4, 4, 17)
(None, 4, 4, 17)
(None, 2, 2, 17)
(None, 2, 2, 17)
(None, 2, 2, 17)
(None, 2, 2, 17)
(None, 2, 2, 15)
(None, 2, 2, 15)
(None, 2, 2, 15)
(None, 2, 2, 36)
(None, 2, 2, 36)
(None, 2, 2, 36)
(None, 1, 1, 36)
[None, 36]
[None, 512]
[None, 512]
[None, 10]
Test accuracy on legitimate examples: 0.1000
Test accuracy on adversarial examples: 0.1000
Test accuracy on legitimate examples: 0.1000
Test accu

KeyboardInterrupt: 

In [ ]:
report.adv_train_adv_eval 

In [ ]:
report.train_adv_train_adv_eval

In [ ]:
# Object used to keep track of (and return) key accuracies
report = AccuracyReport()

# Set TF random seed to improve reproducibility
tf.set_random_seed(1234)


with tf.Session() as sess:
    batch_size = 32

    # Use label smoothing
#     assert Y_train.shape[1] == 10
#     label_smooth = .1
#     Y_train = Y_train.clip(label_smooth / 9., 1. - label_smooth)

    # Define input TF placeholder
    x = tf.placeholder(tf.float32, shape=(None, 32, 32, 3))
    y = tf.placeholder(tf.float32, shape=(None, 10))

    model_path = "models/"
    # Train an MNIST model
#     train_params = {
#         'nb_epochs': nb_epochs,
#         'batch_size': batch_size,
#         'learning_rate': learning_rate
#     }
    fgsm_params = {'eps': 0.3,
                   'clip_min': 0.,
                   'clip_max': 1.}
    rng = np.random.RandomState([2017, 8, 30])

#     if clean_train:
#         model = make_basic_cnn(nb_filters=nb_filters)
#         preds = model.get_probs(x)

#         def evaluate():
#             # Evaluate the accuracy of the MNIST model on legitimate test
#             # examples
#             eval_params = {'batch_size': batch_size}
#             acc = model_eval(
#                 sess, x, y, preds, X_test, Y_test, args=eval_params)
#             report.clean_train_clean_eval = acc
#             assert X_test.shape[0] == test_end - test_start, X_test.shape
#             print('Test accuracy on legitimate examples: %0.4f' % acc)
#         model_train(sess, x, y, preds, X_train, Y_train, evaluate=evaluate,
#                     args=train_params, rng=rng)

        # Calculate training error
#         if testing:
    preds = model.get_probs(x)
    eval_params = {'batch_size': batch_size}
    acc = model_eval(
        sess, x, y, preds, Xtrain, Ytrain, args=eval_params)
    report.train_clean_train_clean_eval = acc

        # Initialize the Fast Gradient Sign Method (FGSM) attack object and
        # graph
        fgsm = FastGradientMethod(model, sess=sess)
        adv_x = fgsm.generate(x, **fgsm_params)
        preds_adv = model.get_probs(adv_x)

        # Evaluate the accuracy of the MNIST model on adversarial examples
        eval_par = {'batch_size': batch_size}
        acc = model_eval(sess, x, y, preds_adv, X_test, Y_test, args=eval_par)
        print('Test accuracy on adversarial examples: %0.4f\n' % acc)
        report.clean_train_adv_eval = acc

        # Calculate training error
        if testing:
            eval_par = {'batch_size': batch_size}
            acc = model_eval(sess, x, y, preds_adv, X_train,
                             Y_train, args=eval_par)
            report.train_clean_train_adv_eval = acc

        print("Repeating the process, using adversarial training")
    # Redefine TF model graph
    model_2 = make_basic_cnn(nb_filters=nb_filters)
    preds_2 = model_2(x)
    fgsm2 = FastGradientMethod(model_2, sess=sess)
    adv_x_2 = fgsm2.generate(x, **fgsm_params)
    if not backprop_through_attack:
        # For the fgsm attack used in this tutorial, the attack has zero
        # gradient so enabling this flag does not change the gradient.
        # For some other attacks, enabling this flag increases the cost of
        # training, but gives the defender the ability to anticipate how
        # the atacker will change their strategy in response to updates to
        # the defender's parameters.
        adv_x_2 = tf.stop_gradient(adv_x_2)
    preds_2_adv = model_2(adv_x_2)

    def evaluate_2():
        # Accuracy of adversarially trained model on legitimate test inputs
        eval_params = {'batch_size': batch_size}
        accuracy = model_eval(sess, x, y, preds_2, X_test, Y_test,
                              args=eval_params)
        print('Test accuracy on legitimate examples: %0.4f' % accuracy)
        report.adv_train_clean_eval = accuracy

        # Accuracy of the adversarially trained model on adversarial examples
        accuracy = model_eval(sess, x, y, preds_2_adv, X_test,
                              Y_test, args=eval_params)
        print('Test accuracy on adversarial examples: %0.4f' % accuracy)
        report.adv_train_adv_eval = accuracy

    # Perform and evaluate adversarial training
    model_train(sess, x, y, preds_2, X_train, Y_train,
                predictions_adv=preds_2_adv, evaluate=evaluate_2,
                args=train_params, rng=rng)

    # Calculate training errors
    if testing:
        eval_params = {'batch_size': batch_size}
        accuracy = model_eval(sess, x, y, preds_2, X_train, Y_train,
                              args=eval_params)
        report.train_adv_train_clean_eval = accuracy
        accuracy = model_eval(sess, x, y, preds_2_adv, X_train,
                              Y_train, args=eval_params)
        report.train_adv_train_adv_eval = accuracy

    return report
